In [122]:
import boto3
import json
import os

aws_resource = "lambda"
state_path = "boardgamegeek.tfstate"
temp_import_file = "temp_import.tf"
state_type = "aws_lambda_function"
output_file = "lambda_imported.tf"
env_var_bucket = ""

client = boto3.client(aws_resource)

create import blocks for Terraform

In [ ]:
repositories = client.list_functions()['Functions']
repositories

In [124]:
names = [f['FunctionName'] for f in repositories]

In [125]:
if os.path.exists(temp_import_file):
    os.remove(temp_import_file)

for name in names:
    import_block = f'resource "{state_type}" "{name}" {{\nname = "{name}"\n}}\n'
    # Write the HCL configuration to a .tf file
    with open(temp_import_file, 'a') as tf_file:
        tf_file.write(import_block)

In [ ]:
os.system("terraform init")

In [ ]:
for name in names:
    import_command = f"terraform import {state_type}.{name} {name}"
    os.system(import_command)

if os.path.exists(temp_import_file):
    os.remove(temp_import_file)

Open imported blocks and extract settings

In [ ]:
data = open(state_path).read()
state = json.loads(data)['resources']
specific_types = [f for f in state if f['type'] == state_type]
specific_types[0]['instances'][0]['attributes']

In [ ]:
# description = "{instance['description']}"

if os.path.exists(output_file):
    os.remove(output_file)

for resource in specific_types[-1:]:
    attributes = resource['instances'][0]['attributes']
    print(resource['name'])

    # environment = attributes['environment']
    # variables = environment[0]['variables'] if len(environment) else {"ENV": "prod"}
    # os.system(f"touch {resource['name']}_lambda.env")
    # with open(f"{resource['name']}_lambda.env", 'w') as f:
    #     for key, value in variables.items():
    #         f.write(f"{key.upper()}={value}\n")
    # print(f"Environment variables written to {resource['name']}_lambda.env")
    # s3client = boto3.client('s3')
    # s3client.upload_file(f"{resource['name']}_lambda.env", '{env_var_bucket}', f"lambda_env_files/{resource['name']}_lambda.env", ExtraArgs={'ContentType': 'text/plain'})
    # os.remove(f"{resource['name']}_lambda.env")

    # Define the HCL configuration for the resource
    if attributes.get("package_type") == "Zip":
        resource_or_image = f'filename = "lambda_function.zip"\n\t\
  runtime = "{attributes["runtime"]}"\n\t\
  handler = "{attributes["handler"]}"'
    elif attributes.get("package_type") == "Image":
        resource_or_image = f'package_type = "{attributes["package_type"]}"\n\timage_uri = "{attributes["image_uri"]}"'
    
    if len(attributes.get("vpc_config")):
        vpc = f'vpc_config {{\n\t\
        subnet_ids = {attributes["vpc_config"][0]["subnet_ids"]}\n\t\
        security_group_ids = {attributes["vpc_config"][0]["security_group_ids"]}\n\t}}'
    else:
        vpc = ""

    description = attributes.get("description", "").replace('"', "").replace(';', "")

    if len(attributes.get("layers")):
        layers = f'layers = {attributes["layers"]}'
    else:
        layers = ""

    hcl = f"""
data "aws_s3_object" "{resource['name']}_lambda_env" {{
  bucket = "hpa-data-eng"
  key = "lambda_env_files/{resource['name']}_lambda.env"
}} 
resource "{state_type}" "{resource['name']}" {{
    depends_on = [data.aws_s3_object.{resource['name']}_lambda_env]
    function_name = "{resource['name']}"
    publish = false
    role = "{attributes['role']}"
    description = "{description}"
    {resource_or_image}
    timeout = {attributes['timeout']}
    {layers}
    tags = {{
        {', '.join([f'"{key}" = "{value}"' for key, value in attributes['tags'].items()])}
    }}
    {vpc}
    environment {{
    variables = {{ for tuple in regexall("(.*?)=(.*)", data.aws_s3_object.{resource['name']}_lambda_env.body) : tuple[0] => tuple[1] }}
  }}
}}
    """.replace("'", '"')


    # Write the HCL configuration to a .tf file
    with open(output_file, 'a') as tf_file:
        tf_file.write(hcl)